### Preparacion

Importar librerías

In [5]:
import pandas as pd
from rdflib import Graph, URIRef, Literal, Namespace
from rdflib.namespace import RDF, XSD, OWL
import requests
import time

### Espacios de nombres & Transformacion a RDF



In [ ]:
def buscar_en_wikidata(nombre_ciudad):
    url = "https://query.wikidata.org/sparql"
    
    # La consulta SPARQL simple
    query = f"""
    SELECT ?item WHERE {{
      ?item rdfs:label "{nombre_ciudad}"@en.
      SERVICE wikibase:label {{ bd:serviceParam wikibase:language "en". }}
    }} LIMIT 1
    """
    
    # Parámetros de la petición GET
    params = {
        'query': query,
        'format': 'json'
    }
    
    # Wikidata exige User-Agent
    headers = {
        'User-Agent': 'PrediccionInundaciones/1.0 (direccion@ejemplo.com)'
    }
    
    try:
        # Hacemos la petición GET (igual que escribir la URL en el navegador)
        r = requests.get(url, params=params, headers=headers)
        data = r.json()
        
        # Navegamos por el JSON de respuesta para sacar el ID (Qxxxx)
        if data['results']['bindings']:
            return data['results']['bindings'][0]['item']['value']
        else:
            return None
    except Exception as e:
        print(f"Error de conexión: {e}")
        return None

# Cache para guardar resultados y no repetir peticiones
cache_ciudades = {}

# --- 2. PREPARACIÓN DEL GRAFO ---
domain = 'https://prediccion-inundaciones.es/'
schema = Namespace("https://schema.org/")

g = Graph()
g.bind("rdf", RDF)
g.bind("xsd", XSD)
g.bind("owl", OWL)
g.bind("schema", schema)

df = pd.read_csv("datos_combinados.csv", sep=';')



print("Procesando datos...")


df.columns = df.columns.str.strip()

for index, row in df.iterrows():
    
    ciudad = str(row['Ciudad']).strip() 
    fecha = str(row['fecha']).strip()
    hora = str(row['hora']).strip()
    
    # place
    city_slug = ciudad.lower().replace(" ", "_")
    place_uri = URIRef(domain + 'location/' + city_slug)
    
    # Consultamos Wikidata si es una ciudad nueva para nosotros
    if ciudad not in cache_ciudades:
        print(f"Buscando '{ciudad}' en Wikidata...")
        uri_wikidata = buscar_en_wikidata(ciudad)
        cache_ciudades[ciudad] = uri_wikidata
        time.sleep(0.5) # pausa entre peticiones
    
    g.add((place_uri, RDF.type, schema.Place))
    g.add((place_uri, schema.name, Literal(ciudad)))
    
    # Si encontramos enlace en Wikidata, lo añadimos (Enriquecimiento)
    if cache_ciudades[ciudad]:
        g.add((place_uri, OWL.sameAs, URIRef(cache_ciudades[ciudad])))

    # event
    timestamp = f"{fecha}T{hora}:00"
    
    # ID único
    safe_time = timestamp.replace("-","").replace(":","").replace("T","_")
    event_id = f"evento_{city_slug}_{safe_time}"
    event_uri = URIRef(domain + 'event/' + event_id)
    
    g.add((event_uri, RDF.type, schema.Event))
    g.add((event_uri, schema.name, Literal(f"Medición en {ciudad}")))
    g.add((event_uri, schema.startDate, Literal(timestamp,
                                             datatype=XSD.dateTime)))
    
    t = row.get('Temperature', 'N/A')
    p = row.get('precipitacion_media', 'N/A')
    c = row.get('Discharge', 'N/A')
    h = row.get('Humidity', 'N/A')
    
    desc = f"Temp: {t}C, Lluvia: {p}mm, Caudal: {c} m3/s, Humedad: {h}%"
    g.add((event_uri, schema.description, Literal(desc)))
    
    # relacion
    g.add((place_uri, schema.event, event_uri))
    g.add((event_uri, schema.location, place_uri))

# guardar
g.serialize(destination="hidrologia_final.ttl", format="turtle")
print("¡Proceso completado!")